In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
sk_less5 = pd.read_csv('Lessthan_5_victim_count.csv')
sk_5to14 = pd.read_csv('5_to_14_victim_count.csv')
sk_15to30 = pd.read_csv('15_to_30_victim_count.csv')
sk_highest = pd.read_csv('Highest_victim_count.csv')
sk = pd.concat([sk_less5,sk_5to14,sk_15to30,sk_highest], axis = 0)
sk.reset_index(drop=True, inplace=True)

First I'm going to check if everything I did in the previous cell is correct.

In [3]:
sk.sample(n=5)
# Looks like everything is fine

,Name,Country,Years active,Proven victims,Possible victims,Notes
262,Atlanta Ripper,United States,1911,15,21,Unidentified serial killer(s) who killed at le...
127,Louis van Schoor,South Africa,1986 to 1989,9,100,Former security guard who was convicted of sev...
86,Benjamin Atkins,United States,1991 to 1992,11,NaN,"Known as ""The Woodward Corridor Killer"". Raped..."
166,Ohio Prostitute Killer,United States,1981 to 2004,7,10,Supposedly murdered prostitutes and exotic dan...
102,Ruslan Khamarov,Ukraine,2000 to 2003,11,NaN,"Seduced, raped and then killed women in his ho..."


Obtaining some info about the dataset is always a good way to start.

In [4]:
sk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              305 non-null    object
 1   Country           305 non-null    object
 2   Years active      305 non-null    object
 3   Proven victims    305 non-null    object
 4   Possible victims  237 non-null    object
 5   Notes             305 non-null    object
dtypes: object(6)
memory usage: 14.4+ KB


All the columns are object type, so I'm going to see what are the unique values for each column and try to clean and create new variables for the later visualization. Also there's a column that has missing values (Possible victims).

### Country

In [5]:
sk.Country.unique()

array(['United States',
       'United Kingdom\r\nIreland (suspected)\r\nWest Germany (suspected)\r\nNetherlands (suspected)\r\nFrance (suspected)',
       'Austria', 'Italy', 'Indonesia', 'Soviet Union', 'United Kingdom',
       'Mexico', 'Allied-occupied Germany\r\nWest Germany', 'Netherlands',
       'Japan', 'Turkey', 'Poland', 'West Germany', 'Brazil', 'Chile',
       'Russia', 'South Korea', 'Pakistan', 'Latvia', 'South Africa',
       'Argentina', 'Soviet Union\r\nRussia', 'Tunisia', 'Germany',
       'Thailand\r\nNepal\r\nIndia\r\nMalaysia', 'Yemen', 'Spain',
       'Australia', 'China', 'Austria\r\nUnited States\r\nCzechoslovakia',
       'Canada', 'France', 'Bangladesh', 'Belgium', 'Ukraine', 'India',
       'Soviet Union\r\nRussia\r\nUkraine',
       'East Germany\r\nGermany\r\nFrance\r\nSpain\r\nCzech Republic (suspected)\r\nItaly (suspected)',
       'German Empire\r\nGermany', 'Denmark', 'China\r\nJapan',
       'Spain\r\nFrance (claimed)\r\nItaly (claimed)',
       'Cana

Since the rows that have more than one country are separated by '\r\n' I will split them and I will count how many of them there are in each row.

In [13]:
sk['Countries_active'] = sk.Country.apply(lambda x: x.split('\r\n'))

In [14]:
sk['Countries_num'] = sk['Countries_active'].apply(lambda x: len(x))

In [15]:
sk.head()

,Name,Country,Years active,Proven victims,Possible victims,Notes,Countries_active,Countries_num
0,Charlie Brandt,United States,1971 to 2004,4,29,Committed suicide by hanging after murdering h...,[United States],1
1,Robert Black,United Kingdom\r\nIreland (suspected)\r\nWest ...,1981 to 1986,4,18+,"Convicted of kidnapping, raping and murdering ...","[United Kingdom, Ireland (suspected), West Ger...",5
2,Max Gufler,Austria,1946 to 1958,4,18,"Poisoned and drowned four women, but suspected...",[Austria],1
3,Ernesto Picchioni,Italy,1949 and earlier,4,16,Murdered people who approached his home; died ...,[Italy],1
4,Baekuni,Indonesia,1993 to 2010,4,14,Pedophile who raped and killed young boys; ini...,[Indonesia],1


### Proven victims

In [16]:
sk['Proven victims'].unique()

array([4, 3, '14', '13', '12', '11–16', '11', '10', '9–13', '9', '8',
       '8–15', '8–11+', '7', '6', '5', 30, 28, 27, 26, 25, 24, 23, 22, 21,
       20, 19, 18, 17, 16, 15, 138, 110, 100, 83, 72, 71, 70, 67, 60, 58,
       53, 52, 50, 49, 48, 45, 42, 41, 39, 38, 36, 35, 33, 32],
      dtype=object)

First of all, if the column is proven victims, why are there some values that are intervals? For those that are intervals I'm going to get the highest number.

In [17]:
sk[(sk['Proven victims'] == '11–16') | (sk['Proven victims'] == '9–13') | (sk['Proven victims'] == '8–15') | (sk['Proven victims'] == '8–11+')]

,Name,Country,Years active,Proven victims,Possible victims,Notes,Countries_active,Countries_num
79,Herb Baumeister,United States,1990 to 1996,11–16,25+,Strangled gay men and buried their bodies in h...,[United States],1
126,Volker Eckert,East Germany\r\nGermany\r\nFrance\r\nSpain\r\n...,1974 to 2006,9–13,19+,German trucker who confessed to having abducte...,"[East Germany, Germany, France, Spain, Czech R...",6
144,Joseph Paul Franklin,United States,1977 to 1980,8–15,20,White supremacist shooter who confessed to 20 ...,[United States],1
145,Yoshio Kodaira,China\r\nJapan,1928? to 1946,8–11+,Unknown,"A serial rapist, Kodaira killed his father-in-...","[China, Japan]",2


In [18]:
sk['Proven_victims'] = sk['Proven victims'].apply(lambda x: max([int(i) for i in str(re.sub('\+$', '', str(x))).split('–')]))

### Possible victims

In [19]:
sk['Possible victims'].unique()

array(['29', '18+', '18', '16', '14', '13', '12', '11+', '4+', '4', '3',
       '50+', '48', '30', '20+', '17', '10', '9–25', '-', '9-100+', '8+',
       '3+', '80+', '21+', nan, '44', '40+', '38+', '20', '16+', '15',
       '13+', '100', '31–80+', '22', '15–16', 'Unknown', '25+', '29–49',
       '17–20', '25', '11', '10+', '19+', '79', '34', '14+', '160',
       '130+', '12+', '50–100', '43+', '23–37', '7', '70?', '15+', '6+',
       '85–100', '37', '17–25', '5–22', '42+', '42', '45', '28', '35+',
       '27+', '26', '24+', '24', '150', '23+', '23', '36+', '26+', '21',
       '82+', '30+', '19', '65–67', '172–300+', '300+', '83+', '180',
       '100+', '70+', '67', '93', '80-300', '56', '52+', '50', '49',
       '71–90+', '60', '60+', '70–80+', '41', '~39', '76', '80–100',
       '43–55+', '~35', '36–100+', '35', '34+', '33', '32+', '32'],
      dtype=object)

This column is a bit more complicated. First for those that have '+', '~' or '?' I will take only the number and delete the rest of the string. In this case I can understand that is possible that the value is an interval because it says possible thus the exact number is not known. I will calculate the mean of the interval. I the number is not exact it will be rounded up to the next int number. 

In [20]:
sk['Possible victims'] = sk['Possible victims'].apply(lambda x: np.nan if ((x == 'Unknown') | (x== '-')) else x)

In [21]:
sk['Possible_victims'] = sk['Possible victims'].apply(lambda x: x if pd.isna(x) else 
                                                      str(re.sub('\+$|^\~|\?', '', str(x))))

In [22]:
sk['Possible_victims'] = sk['Possible_victims'].apply(lambda x: x if pd.isna(x) else 
                             sum([int(i) for i in re.split('–|-',str(x))])/len(str(re.sub('\+$', '', str(x))).split('–'))).round(0)

In [23]:
sk.head()

,Name,Country,Years active,Proven victims,Possible victims,Notes,Countries_active,Countries_num,Proven_victims,Possible_victims
0,Charlie Brandt,United States,1971 to 2004,4,29,Committed suicide by hanging after murdering h...,[United States],1,4,29.0
1,Robert Black,United Kingdom\r\nIreland (suspected)\r\nWest ...,1981 to 1986,4,18+,"Convicted of kidnapping, raping and murdering ...","[United Kingdom, Ireland (suspected), West Ger...",5,4,18.0
2,Max Gufler,Austria,1946 to 1958,4,18,"Poisoned and drowned four women, but suspected...",[Austria],1,4,18.0
3,Ernesto Picchioni,Italy,1949 and earlier,4,16,Murdered people who approached his home; died ...,[Italy],1,4,16.0
4,Baekuni,Indonesia,1993 to 2010,4,14,Pedophile who raped and killed young boys; ini...,[Indonesia],1,4,14.0


### Years active

I wanted to take advantage of this column and I created some more columns.

First I'm going to clean the main column. I'm going to delete those characters that say '?' or 'c.'. After that I'm going to transform those that say 1990s into 1990 to 1999 so it is easier to handle later.

In [17]:
sk['Years active'] = sk['Years active'].apply(lambda x: re.sub('\?|c\.','',str(x)))

In [18]:
# Todos los valores que los pone como década (1990s etc) los he puesto como 1990 to 1999.
sk['Years active'] = sk['Years active'].apply(lambda x: re.sub('s$','', str(x)) + ' to ' + str(int(re.sub('s$','', str(x)))+9) if (len(re.split(' to | and ', str(x)))==1 and 
                                                               len(re.findall('s$', x))==1) else x)

Now I'm going to create two columns, one for the start year and another for the end year. I will split the string separating by ' to ' or ' and '. If the string says 'earlier' (e.g. 1990 and earlier) I will take the year as the finish year and the start year will be unknown. If the string is only one year it will be the start and the finish year.

In [19]:
# Todos los que ponía earlier los he puesto como Unknown
sk['Start_year'] = sk['Years active'].apply(lambda x: 'Unknown' if (len(re.split(' to | and ', str(x)))==2 and 
                         re.split(' to | and ', str(x))[1] == 'earlier') else re.split(' to | and ', str(x))[0])

In [20]:
sk['Finish_year'] = sk['Years active'].apply(lambda x: re.split(' to | and ', str(x))[0] if (len(re.split(' to | and ', str(x)))==2 and 
                         re.split(' to | and ', str(x))[1] == 'earlier') else 
                         (re.split(' to | and ', str(x))[0] if (len(re.split(' to | and ', str(x)))==1) else 
                          re.split(' to | and ', str(x))[1]))

There are some values that are more complicated so I just created a dictionary and replaced them directly.

In [25]:
d1 = {'30 June 1983': '1983', 'late 1990': '1990'}
d2 = {'present': '2023', '23 July 1983': '1983'}
sk['Start_year'] = sk['Start_year'].replace(d1)
sk['Finish_year'] = sk['Finish_year'].replace(d2)

Looks like there are some more values that have 's'. We'll fix them.

In [22]:
sk['Start_year'] = sk['Start_year'].apply(lambda x: re.sub('s','',str(x)))
sk['Finish_year'] = sk['Finish_year'].apply(lambda x: re.sub('s','',str(x)))

Lastly, I will count how many years they were active. I will calculate the difference between the two columns created previously. If one year is Unknown the active years will also be unknown.

In [26]:
sk['Active_years'] = sk.apply(lambda x: (int(x['Finish_year'])-int(x['Start_year'])) if x['Start_year']!= 'Unknown' else
                              'Unknown', axis =1)

In [27]:
sk.head()

,Name,Country,Years active,Proven victims,Possible victims,Notes,Countries_active,Countries_num,Proven_victims,Possible_victims,Start_year,Finish_year,Active_years
0,Charlie Brandt,United States,1971 to 2004,4,29,Committed suicide by hanging after murdering h...,[United States],1,4,29.0,1971,2004,33
1,Robert Black,United Kingdom\r\nIreland (suspected)\r\nWest ...,1981 to 1986,4,18+,"Convicted of kidnapping, raping and murdering ...","[United Kingdom, Ireland (suspected), West Ger...",5,4,18.0,1981,1986,5
2,Max Gufler,Austria,1946 to 1958,4,18,"Poisoned and drowned four women, but suspected...",[Austria],1,4,18.0,1946,1958,12
3,Ernesto Picchioni,Italy,1949 and earlier,4,16,Murdered people who approached his home; died ...,[Italy],1,4,16.0,Unknown,1949,Unknown
4,Baekuni,Indonesia,1993 to 2010,4,14,Pedophile who raped and killed young boys; ini...,[Indonesia],1,4,14.0,1993,2010,17
